In [1]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import numpy as np
import geopandas
import matplotlib.pyplot as plt
from shapely.geometry import Point,Polygon
import csv
import random
from geopandas.tools import sjoin
import multiprocessing as mp
import time

In [2]:
lat = []
lon = []
data_points = 100000
for i in range(data_points):
    x = random.uniform(-90,90)
    lat.append(x)
    
    
for i in range(data_points):
    x = random.uniform(-180,180)
    lon.append(x)

df_cord = pd.DataFrame(lon , columns = ['Lon'])
df_cord['Lat'] = lat
geometry = [Point(xy) for xy in zip(df_cord.Lon, df_cord.Lat)]
df_cord = df_cord.drop(['Lon' , 'Lat' ], axis=1)
crs = {'init': 'epsg:4326'}
gdf = geopandas.GeoDataFrame(df_cord, crs=crs, geometry=geometry)

In [3]:
print(gdf.shape)

(100000, 1)


In [4]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
# ax = world.plot( color='white', edgecolor='black' ,figsize = (40,30))
# gdf.plot(ax=ax, color='red')
# plt.title("World map showing location of given coordinate")
# print(world.head(3))
# plt.show()

In [5]:
df_new = world[['continent' , 'name' , 'geometry']]

In [6]:
def computation(data_split):
    for index,row in data_split.iterrows():
        filter = df_new['geometry'].contains(row['geometry'])
        x = df_new.loc[filter]
        value_c = x['name'].to_string(index = False)
        data_split.loc[index, 'Country'] = value_c if value_c != 'Series([], )' else np.NaN
    return data_split

In [7]:
#computation(gdf)

In [8]:
from pathos.multiprocessing import ProcessingPool as Pool

In [9]:
start_time = time.time()  

cores = 15
partitions = 15


data_split = np.array_split(gdf, cores, axis=0)
pool = Pool(cores)
for i in range(len(data_split)):
    print(data_split[i].shape)
df_out = np.vstack(pool.map(computation, data_split))
pool.close()
pool.join()
pool.clear()
print("--- %s seconds ---" % (time.time() - start_time))

(6667, 1)
(6667, 1)
(6667, 1)
(6667, 1)
(6667, 1)
(6667, 1)
(6667, 1)
(6667, 1)
(6667, 1)
(6667, 1)
(6666, 1)
(6666, 1)
(6666, 1)
(6666, 1)
(6666, 1)
--- 212.1420600414276 seconds ---


In [10]:
op  = pd.DataFrame(data = df_out)
print(op)

                                                   0            1
0       POINT (-63.47758808065264 34.06728956726852)          NaN
1      POINT (-53.00429894148033 -20.76267013040132)       Brazil
2        POINT (35.54866507203954 56.79228919208646)       Russia
3       POINT (-73.34602908537508 53.63442609178307)       Canada
4       POINT (133.0552173636676 -39.54844742628094)          NaN
5       POINT (99.04144404751935 -5.984940654737855)          NaN
6       POINT (-94.38075245515431 -87.7917229232479)   Antarctica
7      POINT (-2.813331133956666 -76.69103684786637)   Antarctica
8       POINT (-32.90066372263988 66.53902313002808)          NaN
9      POINT (-72.84152509283891 -43.75042611245143)        Chile
10     POINT (-127.6635261808017 -11.27898824146122)          NaN
11      POINT (95.66053474529929 -77.12052949301444)   Antarctica
12      POINT (-9.674171851346216 35.37751346177606)          NaN
13       POINT (137.1739763283173 43.87521241907908)          NaN
14      PO